In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3dfe5cb-f9f1-4999-b85f-966544914652;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType, DecimalType, FloatType, LongType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [5]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [6]:
LANDING_PATH= 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher327/prcf/prcf/eih.bnsvher327.prcf.PRCF.CFMVTFAC/'

In [9]:
df = (
    spark
    .read
    .format('json')
    #.schema(schema)
    .load(LANDING_PATH)
    .where(f.col('day') > 2)
)

In [16]:
datetime.fromtimestamp(df.toPandas()['after'][0]['DTEMISSA'] / 1000)

datetime.datetime(2022, 6, 2, 15, 3, 3)

In [14]:
from datetime import datetime

In [17]:
df.toPandas()

after before op                                             source transaction          ts_ms  year  month  day  hour
0     (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918942792179, oracle, CORP1, None, bnsvher327...        None  1654232423620  2022      6    3     2
1     (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918942795029, oracle, CORP1, None, bnsvher327...        None  1654232436981  2022      6    3     2
2     (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918942798713, oracle, CORP1, None, bnsvher327...        None  1654232436996  2022      6    3     2
3     (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918942824439, oracle, CORP1, None, bnsvher327...        None  1654232454710  2022      6    3     2
4     (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918942829060, oracle, CORP1, None, bnsvher327...        None  1654232455636  2022      6    3     2
...                                                 ...    ... ..                                                ...         ...            ...   ...    ...  ...   ...
2122  (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918986873783, oracle, CORP1, None, bnsvher327...        None  1654244377432  2022      6    3     5
2123  (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918993526419, oracle, CORP1, None, bnsvher327...        None  1654246167107  2022      6    3     5
2124  (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918993528982, oracle, CORP1, None, bnsvher327...        None  1654246167110  2022      6    3     5
2125  (BR GO H, 04.347.505/0000-13, 078876950012340,...   None  c  (918993530201, oracle, CORP1, None, bnsvher327...        None  1654246167110  2022      6    3     5
2126  (BR GO H, 02.826.306/0000-11, 078876950012340,...   None  c  (918993544214, oracle, CORP1, None, bnsvher327...        None  1654246167122  2022      6    3     5

[2127 rows x 10 columns]

In [ ]:
df.select('source').printSchema()

In [103]:
df.select('after').columns == df.select('after').columns

True

In [85]:
df.columns

['after',
 'before',
 'op',
 'source',
 'transaction',
 'ts_ms',
 'year',
 'month',
 'day',
 'hour']

In [58]:
df.printSchema()

root
 |-- after: struct (nullable = true)
 |    |-- /AFS/COLLECTION: string (nullable = true)
 |    |-- /AFS/THEME: string (nullable = true)
 |    |-- AENAM: string (nullable = true)
 |    |-- CEINM: string (nullable = true)
 |    |-- CHARG: string (nullable = true)
 |    |-- CHDLL: string (nullable = true)
 |    |-- CHJIN: string (nullable = true)
 |    |-- CHRUE: string (nullable = true)
 |    |-- CINSM: string (nullable = true)
 |    |-- CLABS: string (nullable = true)
 |    |-- CRETM: string (nullable = true)
 |    |-- CSPEM: string (nullable = true)
 |    |-- CUMLM: string (nullable = true)
 |    |-- CVMEI: string (nullable = true)
 |    |-- CVMIN: string (nullable = true)
 |    |-- CVMLA: string (nullable = true)
 |    |-- CVMRE: string (nullable = true)
 |    |-- CVMSP: string (nullable = true)
 |    |-- CVMUM: string (nullable = true)
 |    |-- ERNAM: string (nullable = true)
 |    |-- ERSDA: string (nullable = true)
 |    |-- HERKL: string (nullable = true)
 |    |-- J_3ABZUS:

In [71]:
(
    df
    .limit(10)
    .toPandas()
)['after'][0]

Row(/AFS/COLLECTION=' ', /AFS/THEME=' ', AENAM='HRNG_JOBS_SD', CEINM='0.000', CHARG='0000414678', CHDLL='20191023', CHJIN='2019', CHRUE='X', CINSM='0.000', CLABS='0.000', CRETM='0.000', CSPEM='0.000', CUMLM='0.000', CVMEI='0.000', CVMIN='0.000', CVMLA='0.000', CVMRE='0.000', CVMSP='0.000', CVMUM='0.000', ERNAM='VBDIAS', ERSDA='20160128', HERKL=' ', J_3ABZUS=' ', J_3ARESM='0.000', J_3ASEAN=' ', J_3ASIZE='P', J_3AUANR=' ', J_3AUEPS='0000', J_3AUPOS='000000', J_4KSCAT='BR    0', KZICE=' ', KZICL='  X', KZICQ=' ', KZICS='  X', KZVCE=' ', KZVCL='  X', KZVCQ=' ', KZVCS=' ', LAEDA='20211202', LFGJA='2021', LFMON='12', LGORT='0602', LVORM=' ', MANDT='400', MATNR='6HW71ASN', SPERC=' ', WERKS='H110')

In [40]:
@f.udf(returnType=DoubleType())
def big_decimal_to_decimal(big_decimal):
    bytes_val = base64.decodebytes(big_decimal.encode())
    bval = "".join("{0:08b}".format(c) for c in bytes_val)
    intval = int(bval, 2)

    return intval/(10**2)


In [84]:
(
    df
    .limit(10)
    .toPandas()
)['after'][0]

Row(/AFS/COLLECTION=' ', /AFS/THEME=' ', AENAM='HRNG_JOBS_SD', CEINM='0.000', CHARG='0000414678', CHDLL='20191023', CHJIN='2019', CHRUE='X', CINSM='0.000', CLABS='0.000', CRETM='0.000', CSPEM='0.000', CUMLM='0.000', CVMEI='0.000', CVMIN='0.000', CVMLA='0.000', CVMRE='0.000', CVMSP='0.000', CVMUM='0.000', ERNAM='VBDIAS', ERSDA='20160128', HERKL=' ', J_3ABZUS=' ', J_3ARESM='0.000', J_3ASEAN=' ', J_3ASIZE='P', J_3AUANR=' ', J_3AUEPS='0000', J_3AUPOS='000000', J_4KSCAT='BR    0', KZICE=' ', KZICL='  X', KZICQ=' ', KZICS='  X', KZVCE=' ', KZVCL='  X', KZVCQ=' ', KZVCS=' ', LAEDA='20211202', LFGJA='2021', LFMON='12', LGORT='0602', LVORM=' ', MANDT='400', MATNR='6HW71ASN', SPERC=' ', WERKS='H110')

In [72]:
df_table = df

In [73]:
cols = ('before', 'source')
df_table = df_table.drop(*cols)
df_table = df_table.withColumnRenamed("ts_ms", "timestamp_kafka").withColumnRenamed("op", "__op")

In [18]:
df_table

NameError: name 'df_table' is not defined

In [44]:
flatten_df(df_table).toPandas()

__op transaction  timestamp_kafka  year  month  day  hour /AFS/COLLECTION /AFS/THEME         AENAM  CEINM       CHARG     CHDLL CHJIN CHRUE  CINSM  CLABS  CRETM  CSPEM  CUMLM  CVMEI  CVMIN  CVMLA  CVMRE  CVMSP  CVMUM         ERNAM     ERSDA HERKL J_3ABZUS J_3ARESM J_3ASEAN J_3ASIZE J_3AUANR J_3AUEPS J_3AUPOS   J_4KSCAT KZICE KZICL KZICQ KZICS KZVCE KZVCL KZVCQ KZVCS     LAEDA LFGJA LFMON LGORT LVORM MANDT       MATNR SPERC WERKS
0    r        None    1653515438099  2022      5   25    18                             HRNG_JOBS_SD  0.000  0000414678  20191023  2019     X  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000        VBDIAS  20160128                   0.000                 P              0000   000000    BR    0           X           X           X              20211202  2021    12  0602         400    6HW71ASN        H110
1    r        None    1653515438099  2022      5   25    18                             HRNG_JOBS_SD  0.000  0000414669  20170831  2017     X  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000        VBDIAS  20160128                   0.000                 P              0000   000000    BR    0                       X           X              20180609  2018    06  0602         400    6QMY1ASN        H110
2    r        None    1653515438099  2022      5   25    18                             HRNG_JOBS_SD  0.000  0000414671  00000000  2016        0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000        VBDIAS  20160128                   0.000                 G              0000   000000    BR    0                                                  20180604  2018    06  0602         400    6QMY1ASN        H110
3    r        None    1653515438099  2022      5   25    18                             HRNG_JOBS_SD  0.000  0000414672  00000000  2016        0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000        VBDIAS  20160128                   0.000                XG              0000   000000    BR    0                                                  20161221  2016    12  0602         400    6QMY1ASN        H110
4    r        None    1653515438099  2022      5   25    18                                  RFCUSER  0.000  0000414441  00000000  2016     X  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000       RFCUSER  20160127                   0.000               001              0000   000000    BR SC H           X                                      20160226  2016    02  0600         400   C59PJEJMG     A  H102
5    r        None    1653515438099  2022      5   25    18                                  RFCUSER  0.000  0000414532  20160411  2016     X  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000       RFCUSER  20160127                   0.000                 M              0000   000000    BR SC H           X                                      20160226  2016    02  0600         400  H2BLP4SE01        H081
6    r        None    1653515438099  2022      5   25    18                                    ESANI  0.000  0000415437  00000000  2016        0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000         RLIMA  20160128                   0.000                 M              0000   000000  BR GO H E                                                  20160411  2016    04  0608         400  4DT0XKU10S        H002
7    r        None    1653515438099  2022      5   25    18                                    ESANI  0.000  0000415438  00000000  2016        0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000         RLIMA  20160128                   0.000                 G              0000   000000  BR GO H E                                                  20160411  2016    04  0608         400  4DT0XKU10S        H002
8    r        None    1653515438099  2022      5   25    18                            

In [ ]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [f.col(nc+'.'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [145]:
def big_decimal_to_decimal(big_decimal, scale, precision):
    bytes_val = base64.decodebytes(big_decimal.encode())
    bval = "".join("{0:08b}".format(c) for c in bytes_val)
    intval = int(bval, 2)
    if bytes_val[0] & 0x70 != 0:
        intval -= int('1' + '00' * len(bytes_val), precision)
    return intval/(10**2)

In [187]:
import decimal
import base64
for i in range(2,30):


    print(str(i) + ' | ' + str(big_decimal_to_decimal('G1c=', 2, i)))

2 | 69.99
3 | 69.99
4 | 69.99
5 | 69.99
6 | 69.99
7 | 69.99
8 | 69.99
9 | 69.99
10 | 69.99
11 | 69.99
12 | 69.99
13 | 69.99
14 | 69.99
15 | 69.99
16 | 69.99
17 | 69.99
18 | 69.99
19 | 69.99
20 | 69.99
21 | 69.99
22 | 69.99
23 | 69.99
24 | 69.99
25 | 69.99
26 | 69.99
27 | 69.99
28 | 69.99
29 | 69.99


In [ ]:
big_decimal_to_decimal('Thw=', 2, 16)

187.0

In [ ]:
big_decimal_to_decimal

<function __main__.big_decimal_to_decimal(big_decimal, scale, precision)>

In [123]:
coded_string = "Thw="
bytes_val = base64.decodebytes(coded_string.encode())
bval = "".join("{0:08b}".format(c) for c in bytes_val)
intval = int(bval, 2)
if bytes_val[0] & 0x70 != 0:           # manually takes the two's complement
    intval -= int('1' + '00' * len(bytes_val), 16)
print(intval)

-45540


In [171]:
type(df.limit(10).toPandas()['receita_camisetas'][0])

bytearray

In [146]:
import decimal
import base64
def big_decimal_to_decimal(big_decimal, scale, precision):
    bytes_val = base64.decodebytes(big_decimal.encode())
    bval = "".join("{0:08b}".format(c) for c in bytes_val)
    intval = int(bval, 2)
    if bytes_val[0] & 0x70 != 0:
        intval -= int('1' + '00' * len(bytes_val), 16)
    return intval/(10**scale)

In [165]:
big_decimal_to_decimal(bytes(df.limit(10).toPandas()['receita_camisetas'][4]).decode('utf-8'), 2, 12)

ValueError: invalid literal for int() with base 2: ''

In [145]:
(
    df
    .withColumn('receita', f.col("receita").cast("STRING"))
    .select(f.col('receita'))
    .toPandas()
)

[0]
[127, 45]


receita
0      ]�
1        
2        
3        
4     8
5        
6        
7        
8     
9      ��

In [135]:
json.loads(df.limit(10).toPandas()['receita_camisetas'][5])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [132]:
def decoder(s):
    if s is None:
        return None

    loaded_json = json.loads(s.decode())
    loaded_json["ID"] = float.from_bytes(base64.b64decode(loaded_json['ID']), "big")
    return loaded_json

In [27]:
(
    df
    .withColumn('tmc', f.col("tmc").cast("float"))
    #.withColumn('receita', f.col("receita").cast("float"))
    .withColumn('pecas', f.col("pecas").cast("float"))
    .withColumn('dt_registro', f.col("dt_registro").cast("timestamp"))
    .limit(10)
    .select(f.col('tmc'))
    #.select(f.col('receita'))
    .select(f.col('pecas'))
    .select(f.col('dt_registro'))
    .toPandas()
)

AnalysisException: cannot resolve '`pecas`' given input columns: [tmc];
'Project ['pecas]
+- Project [tmc#2551]
   +- GlobalLimit 10
      +- LocalLimit 10
         +- Project [__deleted#2359, __op#2360, __source_ts_ms#2361L, __table#2362, canal#2363, celular#2364, cep#2365, ciclo#2366, cidade#2367, cluster#2368, clv#2369, cpf_cnpj_cliente#2370, cpf_cnpj_cliente_hash#2371, customer_id#2372, data_ult_engajamento#2373, dias_ult_engajamento#2374, dpc#2375L, dt_cad_cliente#2376L, dt_nasc_cliente#2377L, cast(dt_registro#2378L as timestamp) AS dt_registro#2745, duc#2379L, email#2380, flg_aceite_politica#2381L, flg_black_list_email#2382L, flg_black_list_sms#2383L, flg_black_list_whatsapp#2384L, flg_camiseta_basica#2385L, flg_cel_valido#2386L, flg_cliente_marca_ativo#2387, flg_email_valido#2388L, flg_endereco_valido#2389L, flg_funcionario#2390L, flg_moletom#2391L, flg_moletom_cor#2392, flg_omni#2393L, flg_pf#2394L, flg_promo#2395L, flg_quarentena#2396L, flg_shop_fem_fam#2397, flg_shop_fem_inf#

In [49]:
last_date, snapshot_flag = last_date(BRONZE_PATH_PRODUTOS, spark)

In [50]:
df_table = get_data(spark, LANDING_PATH_PRODUTOS, 'timestamp', last_date, 'json', True)

In [51]:
df_table.limit(2).toPandas()

key  \
0  [123, 34, 115, 99, 104, 101, 109, 97, 34, 58, ...   
1  [123, 34, 115, 99, 104, 101, 109, 97, 34, 58, ...   

                                               value  \
0  [123, 34, 115, 99, 104, 101, 109, 97, 34, 58, ...   
1  [123, 34, 115, 99, 104, 101, 109, 97, 34, 58, ...   

                                       topic  partition  offset  \
0  eih.bnsvher602.linx_db.dbo.PRODUTOS_BARRA          0       0   
1  eih.bnsvher602.linx_db.dbo.PRODUTOS_BARRA          0  477741   

                timestamp  year  month  day  hour  
0 2022-05-12 23:26:38.868  2022      5   12    23  
1 2022-05-12 23:27:47.165  2022      5   12    23

In [52]:
df_table = df_table.withColumnRenamed("timestamp", "timestamp_kafka").withColumn('fixedValue', f.col("value").cast("string"))

In [53]:
schema= get_schema(df_table) 
schema

StructType(List(StructField(CODIGO_BARRA,StringType,true),StructField(PRODUTO,StringType,true),StructField(COR_PRODUTO,StringType,true),StructField(TAMANHO,StringType,true),StructField(GRADE,StringType,true),StructField(TIMESTAMP,StringType,true),StructField(DATA_PARA_TRANSFERENCIA,StringType,true),StructField(NOME_CLIFOR,StringType,true),StructField(CODIGO_BARRA_PADRAO,StringType,true),StructField(inativo,StringType,true),StructField(HR_TRIBUT_ORIGEM,StringType,true),StructField(HR_TRIBUT_ORIGEM_FAB,StringType,true),StructField(LX_HASH,StringType,true),StructField(TIPO_COD_BAR,StringType,true),StructField(SKU,StringType,true),StructField(PRODUCTCODE,StringType,true),StructField(__op,StringType,true),StructField(__table,StringType,true),StructField(__source_ts_ms,StringType,true),StructField(__deleted,StringType,true)))

In [54]:
df_temp = decode_data_frame(df_table,schema)

In [55]:
df_table_flat = flatten_df(df_temp)

In [56]:
df_table_flat.limit(10).toPandas()

CODIGO_BARRA       PRODUTO COR_PRODUTO TAMANHO GRADE  \
0  7805810109010              MC34          0000             1   UNI   
1              7909588117458  M5M4          1AEN             3   M     
2  7805810155093              MCIT          0000             5    XG   
3              7909588117472  M5M4          1AEN             5   XG    
4  7805810216862              MC40          MARINHO          4     G   
5              7909588117496  M5M4          1BEN             3   M     
6  7805810257544              MCMO          FUME             2     P   
7              7909588117557  M5M4          1CEN             5   XG    
8  7805810257803              MCMO          PRETO            3     M   
9              7909588117564  M5M4          1DEN             2   P     

      TIMESTAMP DATA_PARA_TRANSFERENCIA NOME_CLIFOR CODIGO_BARRA_PADRAO  \
0  AAAAArC4auI=           1299846568070        None               false   
1  AAAAArDOTrI=           1553171560517        None               false   
2  AAAAArC4auU=           1117646407063        None               false   
3  AAAAArDOTrQ=           1553171560517        None               false   
4  AAAAArC4auw=           1117646407063        None               false   
5  AAAAArDOTrY=           1553171560517        None               false   
6  AAAAArC4au4=           1113468100877        None               false   
7  AAAAArDOTrw=           1553171560517        None               false   
8  AAAAArC4avA=           1117646407063        None               false   
9  AAAAArDOTr0=           1553171560517        None               false   

  inativo  ... HR_TRIBUT_ORIGEM_FAB LX_HASH TIPO_COD_BAR           SKU  \
0   false  ...                  0      None            1     MC3400001   
1   false  ...                  0      None            1     M5M41AEN3   
2   false  ...                  0      None            1     MCIT00005   
3   false  ...                  0      None            1     M5M41AEN5   
4   false  ...                  0      None            1  MC40MARINHO4   
5   false  ...                  0      None            1     M5M41BEN3   
6   false  ...                  0      None            1     MCMOFUME2   
7   false  ...                  0      None            1     M5M41CEN5   
8   false  ...                  0      None            1    MCMOPRETO3   
9   false  ...                  0      None            1     M5M41DEN2   

    PRODUCTCODE __op         __table __source_ts_ms __deleted  \
0   MC340000UNI    r  PRODUTOS_BARRA  1652397997504     false   
1     M5M41AENM    r  PRODUTOS_BARRA  1652398066635     false   
2    MCIT0000XG    r  PRODUTOS_BARRA  1652397997505     false   
3    M5M41AENXG    r  PRODUTOS_BARRA  1652398066635     false   
4  MC40MARINHOG    r  PRODUTOS_BARRA  1652397997505     false   
5     M5M41BENM    r  PRODUTOS_BARRA  1652398066635     false   
6     MCMOFUMEP    r  PRODUTOS_BARRA  1652397997505     false   
7    M5M41CENXG    r  PRODUTOS_BARRA  1652398066635     false   
8    MCMOPRETOM    r  PRODUTOS_BARRA  1652397997505     false   
9     M5M41DENP    r  PRODUTOS_BARRA  1652398066635     false   

          timestamp_kafka  
0 2022-05-12 23:26:38.868  
1 2022-05-12 23:27:47.165  
2 2022-05-12 23:26:38.869  
3 2022-05-12 23:27:47.165  
4 2022-05-12 23:26:38.870  
5 2022-05-12 23:27:47.165  
6 2022-05-12 23:26:38.870  
7 2022-05-12 23:27:47.166  
8 2022-05-12 23:26:38.870  
9 2022-05-12 23:27:47.166  

[10 rows x 21 columns]

In [57]:
df_table = (
    spark
    .read
    .format('delta')
    .load(SILVER_PATH_PRODUTOS)
)

AnalysisException: `gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/produtos_barra` is not a Delta table.

In [22]:
df_table.where(f.col('__op') != 'r').toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


PRODUTO CODIGO_PRECO MATERIAL PERIODO_PCP TABELA_OPERACOES  \
0     06AK                          None        None             None   
1     54NA                          None        None             None   
2     54NL                          None        None             None   
3     54RR                          None        None             None   
4     55WU                          None        None             None   
...            ...          ...      ...         ...              ...   
1828  58N6                          None        None             None   
1829  7CL5                          None        None             None   
1830  KFKN                          None        None             None   
1831  KZTR                          None        None             None   
1832  KZVW                          None        None             None   

     FATOR_OPERACOES CLASSIF_FISCAL TIPO_PRODUTO TABELA_MEDIDAS  \
0               A+g=     61102000        MOLETOM           None   
1               A+g=     61102000        MOLETOM           None   
2               A+g=     61012000        MOLETOM           None   
3               A+g=     61102000        MOLETOM           None   
4               A+g=     61034200          MALHA           None   
...              ...            ...          ...            ...   
1828            A+g=     61142000          MALHA           None   
1829            A+g=     61083100          MALHA           None   
1830            A+g=     61152920         OUTROS           None   
1831            A+g=     61046900         PESADO           None   
1832            A+g=     61102000         PESADO           None   

              DESC_PRODUTO  ... DT_HRYBRIS_ULT_ATUALIZACAO B2C_CONSERVACAO  \
0            BLUSAO ML FEM  ...              1632597840673            None   
1           BLUSAO ML MASC  ...              1634498803697            None   
2             JAQUETA MASC  ...              1621881220560            None   
3           BLUSAO ML MASC  ...              1634498803697            None   
4             BERMUDA MASC  ...                       None            None   
...                    ...  ...                        ...             ...   
1828               BODY MM  ...                       None            None   
1829         PIJAMA MM FEM  ...              1625682057953            None   
1830          MEIA UNISSEX  ...              1634413778543            None   
1831  CALCA PANTACOURT FEM  ...              1634499493477            None   
1832           BLUSAO MASC  ...                       None            None   

     HR_B2C_CATEGORIA ID_CEST_NCM HR_INDICA_BRINDE HR_PARCEIRO HR_ESTRATEGIA  \
0                None        None             None        None          None   
1            INFANTIL        None             None        None          None   
2            INFANTIL        None             None        None          None   
3            INFANTIL        None             None        None          None   
4            INFANTIL        None             None        None          None   
...               ...         ...              ...         ...           ...   
1828         INFANTIL        None             None        None          None   
1829             None        None             None        None          None   
1830             None        None             None        None          None   
1831         INFANTIL        None             None        None          None   
1832         INFANTIL        None             None        None          None   

     INFO_MODELO __op         timestamp_kafka  
0           None    u 2022-05-14 04:43:02.080  
1           None    u 2022-05-14 09:31:41.371  
2           None    u 2022-05-14 07:59:20.202  
3           None    u 2022-05-14 09:25:40.480  
4           None    u 2022-05-14 08:25:58.583  
...          ...  ...                     ...  
1828        None    u 2022-05-14 09:05:14.304  
1829        None    u 2022-05-14 07:27:31.226  

In [ ]:
teste = (
    spark
    .read
    .format('json')
    .schema(schema)
    .load(LANDING_PATH_PRODUTOS)
)

In [ ]:
teste.limit(2).toPandas()

In [ ]:
teste = teste.withColumnRenamed("timestamp", "timestamp_kafka").withColumn('fixedValue', f.col("value").cast("string"))

In [ ]:
json_fields

In [ ]:
def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

In [ ]:
schema2 = get_schema(teste) 

In [ ]:
schema2

In [ ]:
json_str = teste.limit(1).toPandas()['fixedValue'][0]
json_str

In [ ]:
json_fields = json.loads(json_str)['schema']['fields']

In [ ]:
json_fields = json.loads(json_str)['schema']['fields']

In [ ]:
schema= get_schema(teste) 

In [ ]:
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

In [ ]:
df_bronze = (
    spark
    .read
    .format("json")
    .load(landing_path)
)

In [ ]:
df_bronze.count()

In [ ]:
# Reading last bronze data
try:
    df_bronze = (
        spark
        .read
        .format("delta")
        .load(bronze_path)
    )

    list_result = (
        df_bronze
        .agg({"timestamp_kafka": "max"}).collect()
    )
    
    last_date = list_result[0].asDict()['max(timestamp_kafka)']
    
except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [ ]:
print(last_date)

In [ ]:
# Reading landing data
df_table = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp")>last_date)
    .withColumnRenamed("timestamp", "timestamp_kafka")
)

In [ ]:
df_table.toPandas()

In [ ]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [ ]:
df_table_flat = flatten_df(df_temp)

In [ ]:
(
    df_table_flat
    .write
    .partitionBy("year", "month", "day", "hour")
    .format("delta")
    .mode("append")
    .option("checkpointLocation", bronze_path + "checkpoint")
    .save(bronze_path)
)

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

In [ ]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)